In [1]:
import torch
import transformers
import peft


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /home/lxe/miniconda3/envs/llama-finetuner/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/lxe/miniconda3/envs/llama-finetuner/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


In [4]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    'cerebras/Cerebras-GPT-2.7B', 
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map='auto'
)

In [5]:
model.config

GPT2Config {
  "_name_or_path": "cerebras/Cerebras-GPT-2.7B",
  "activation_function": "gelu",
  "attn_pdrop": 0.0,
  "bos_token_id": 50256,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_embd": 2560,
  "n_head": 32,
  "n_inner": 10240,
  "n_layer": 32,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.0,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float16",
  "transformers_version": "4.28.0.dev0",
  "use_cache": true,
  "vocab_size": 50257
}

In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained('decapoda-research/llama-7b-hf')
# tokenizer.pad_token_id = 0

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [9]:
model = peft.PeftModel.from_pretrained(
    model,
    'lora-assistant',
    torch_dtype=torch.float16
)

In [9]:
inputs = tokenizer("Human: How old is the sun?\nAssistant:", return_tensors="pt")
input_ids = inputs["input_ids"].to('cuda')

generation_config = transformers.GenerationConfig(
    do_sample = True,
    temperature = 0.3,
    top_p = 0.1,
    top_k = 80,
    repetition_penalty = 1.5,
    max_new_tokens = 50
)

with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config,
    )
    
output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
print(output_text)

Human: How old is the sun?
Assistant: I'm not a member.
I am you, but it was and that he's.


In [6]:
del model